# Planes of the Sky

Brett Deaton -- Dec 2020

#### From Explicit Definitions

See Wikipedia's
[celestial coordinate system](https://en.wikipedia.org/wiki/Celestial_coordinate_system)
and Binney & Merrifield, *Galactic Astronomy*, 1998.

In [ ]:
from math import sin, cos, tan, pi, asin, acos, atan
import numpy as np
np.set_printoptions(suppress=True)
class SkyPosition:
    """Sky position in celestial equatorial coordinates.
    
    In addition to equatorial coordinates (alpha, delta) used to initialize,
    we also use ecliptic (lambda, beta) and galactic (ell, bee) coordinates,
    where the two arguments are ordered (long, lat), and all angles are degrees
    except right ascension, alpha.
    
    Args (floats):
        alpha -- right ascension [hours], def=0
        delta -- declination [degrees], def=0
    """
    def __init__(self, alpha=0, delta=0):
        # store colatitude theta and azimuth phi in rad
        self.alpha = alpha*pi/12
        self.delta = delta*pi/180
        self.theta = (90-delta)*pi/180   # colatitude
        self.phi = self.alpha            # azimuth
        self.EPS = 23.44*pi/180          # obliquity of ecliptic
        self.ALPHANGP = 192.86*pi/180    # right ascension galactic pole
        self.DELTANGP = 27.13*pi/180     # declination galactic pole
        self.ELLNCP = 122.93*pi/180      # galactic longitude celestial pole
    def unitn(self):
        """Return unit 3-vector as numpy array."""
        st = sin(self.theta)
        n = [st*cos(self.phi), st*sin(self.phi), cos(self.theta)]
        return np.array(n)
    def eclipticAngles(self):
        ca = cos(self.alpha)
        sa = sin(self.alpha)
        cd = cos(self.delta)
        sd = sin(self.delta)
        td = tan(self.delta)
        ce = cos(self.EPS)
        se = sin(self.EPS)
        lamb = atan((sa*ce+td*se)/ca)*180/pi
        beta = asin(sd*ce-cd*se*sa)*180/pi
        return (lamb, beta)

In [ ]:
# Example, the four cardinal positions on the celestial equator
nEqus = [SkyPosition(hr).unitn() for hr in range(0,24,6)]
posNames = ["VE", "SS", "AE", "WS"] # vernal equinox, summer solstice, etc
print("## Cardinal positions (equatorial coords)")
for name, n in zip(posNames, nEqus):
    print(name, ": ", n, sep="")

In [ ]:
# Entering-boundary of the constellations along the ecliptic, i.e.
# the right ascension of the sun when it enters that constellation.
# Epoch J2000.0, from Norton's J2000.0 star charts, by eye.
# Tuples are (right ascension [hrs], declination [deg]).
constel = {
    "Ari": (1.8, 11),
    "Tau": (3.4, 18),
    "Gem": (6.0, 23),
    "Can": (8.0, 21),
    "Leo": (9.3, 15),
    "Vir": (11.6, 2),
    "Lib": (14.4, -4),
    "Sco": (15.9, -20),
    "Oph": (16.4, -22),
    "Sag": (17.7, -24),
    "Cap": (20.1, -20),
    "Aqu": (22.0, -12),
    "Pis": (23.5, -3),
}

In [ ]:
anglesEcl = [SkyPosition(*p).eclipticAngles() for p in constel.values()]
print("## Entering angles of sun (ecliptic longitude, deg)")
for name, a in zip(constel.keys(), anglesEcl):
    print(name, ": ", round(a[0]*12/180,1), sep="")

## Todo

The ecliptic longitudes are wrong above. Probably need to clarify the
trig branching for atan, asin, etc.

## Scratch

#### From Rotation Matrices

In [ ]:
class RotMatrix:
    """Rotation matrices between celestial coordinate systems, numpy arrays.
    
    Specifies a coordinate transformation from representation of 3-vector in
    one coordinate system to the same in a new system, n->n', via n'=Rn.
    The three coordinate systems are equatorial (Equ), ecliptic (Ecl), and
    galactic (Gal), giving rise to 6 possible transformation matrices,
    R, e.g. EquToEcl, etc.
    """
    def __init__(self):
        self.eps = 23.44*pi/180 # obliquity of the ecliptic
        ce = cos(self.eps)
        se = sin(self.eps)
        self.EquToEcl = np.array([[1,   0,   0],
                                  [0,  ce,  se],
                                  [0, -se,  ce]])
        self.EclToEqu = np.array([[1,   0,   0],
                                  [0,  ce, -se],
                                  [0,  se,  ce]])

In [ ]:
# Example, the four cardinal positions on the celestial equator
r = RotMatrix()
nEcl = [r.EquToEcl.dot(n) for n in nEqus]
print("## Cardinal positions (ecliptic coords)")
for name, n in zip(posNames, nEcl):
    print(name, ": ", n, sep="")

In [ ]:
print(SkyPosition(*constel["Ari"]).unitn())

In [ ]:
# Compute the position of the sun as it enters each constellation
pEqu = [SkyPosition(*p).unitn() for p in constel.values()]
print("## Entering position of sun (equatorial coords)")
for name, n in zip(constel.keys(), pEqu):
    print(name, ": ", n, sep="")